In [4]:
from cryptography.fernet import Fernet
import pyAesCrypt
import pickle
import os,sys,stat
import getpass
import re

class Password(object):
    
    
    def __init__(self):
        self.__password=None
        self.__filename ='bin'
        
    def load_password(self,psw):
        infile = open(self.__filename,'rb')
        lst = pickle.load(infile)
        infile.close()
        cipher_suite = Fernet(lst[1])
        unciphered_text = (cipher_suite.decrypt(lst[0]))
        self.__password=unciphered_text.decode("utf-8")
        if self.key==psw:
            return(True)
        else:
            self.__password=None
            return(False)
        
    def set_password(self):
        key = Fernet.generate_key()
        cipher_suite = Fernet(key)
        self.__password=getpass.getpass("ENTERT PASSWORD : ")
        password1=bytes(self.key,"utf-8")
        ciphered_text = cipher_suite.encrypt(password1) 
        lst=[ciphered_text,key]
        outfile = open(self.__filename,'wb')
        pickle.dump(lst,outfile)
        outfile.close()
        
    def getPassword(self):
        return(self.__password)
    key=property(getPassword)
        
        
              
class Encryption(object):
    
    @staticmethod
    def Files(file):
        lst=[]
        counter=1
        for r, d, f in os.walk(file):
                for i in f:
                    print(r+'\\'+i)
                    lst.append(r+'\\'+i) #if posix then replace '\\' with '/'
                    counter+=1
                break    
        return lst               
                        
    def __init__(self,password=None):
        self.__password=password
        
    def get_files(self):  
        
        print("ENTER FILES NAME :")
        lst=[]
        counter=1
        while(1):
            file=(input('{}.file :- '.format(counter)))
            if(file==''):
                break
            if(os.path.isdir(file)):
                print("its a directory\ngetting all the files...")
                lst.extend(self.Files(file))
            else:    
                lst.append(file)
            counter+=1     
        return lst
        
        
        
    def start_encryption(self,listOfFiles,path):
        bufferSize=1024*8
        print("wait..")
        ch=int(input("1.keep original\n2.delete original\n"))
        try:
            for i in listOfFiles:
                with open(i,'rb') as fin:
                    f1=i.split('\\')                                               #if posix then replace '\\' with '/'
                    f2=f1[len(f1)-1]
                    with open('{}\\{}.aes'.format(path,f2),'wb') as fout:
                        print(i)
                        pyAesCrypt.encryptStream(fin,fout,self.__password.key,bufferSize,)
                        fin.close()
                        fout.close()
            if(ch == 2):
                for i in listOfFiles:
                    os.remove(i)
            print("SUCCESSFUL")
        except Exception as e:
            print(e)   
       
    
class Decryption(object):               
                        
    def __init__(self,password=None):
        self.__password=password
        self.lst=[]
        
    def get_files(self,path):
        counter=1
        for r, d, f in os.walk(path):
            for i in f:
                if(re.search(".aes$",i) !=None):
                    print("Files {}. :{}".format(counter,r+'\\'+i))         #if posix then replace '\\' with '/'
                    self.lst.append(r+'\\'+i)
                    counter+=1
            break        
        ch=int(input("want to decrypt all files..\n1.YES\n2.NO\n"))
        if(ch==2):
            counter=1
            self.lst=[]
            while(1):
                file=input("{}.file:- ".format(counter))
                if(file==''):
                    break
                self.lst.append(file)
                counter+=1

            
    def start_decrypt(self):
        print("wait...")
        try:
            bufferSize=1024*8
            for i in self.lst:
                encfileSize=os.stat(i).st_size
                with open(i,'rb') as fin:
                    with open(i.rstrip('.aes'),'wb') as fout:
                        print(i)
                        pyAesCrypt.decryptStream(fin,fout,self.__password.key,bufferSize,encfileSize)
                        fin.close()
                        os.remove(i)
                        fout.close()
            return("SUCCESS")  
            
        except Exception as e:
            return(e)

def option():
    print("*"*27)
    print("WELCOME {}".format(getpass.getuser()))
    print("*"*27)
    print('_'*10,'MENU','_'*10)
    ch=int(input("1.Encrypt File\n2.Decrypt File\n3.Change Password\n"))
    return ch
def change_pass():
    print("*"*24)
    print("WELCOME :{}".format(getpass.getuser()))
    input("Press any key to continue")
    print("set new password".upper())
    print('*'*24)
    psw=Password()
    psw.set_password()
    print("Note:\n1.Don not delete 'bin' file if you have any encrypted file ")    
    
def main():
    if(os.path.exists("bin")==True):

        try:
            psw=Password()
            pscd=getpass.getpass("ENTER PASSWORD:- ")
            if(psw.load_password(pscd)==True):
                ch=option()
                if(ch==1):
                    encrypt=Encryption(psw)
                    lst=encrypt.get_files()
                    print(lst)
                    path=input("Enter path to save the encrypted files:- ")
                    encrypt.start_encryption(lst,path)
                elif(ch==2):
                    decypt=Decryption(psw)
                    directory=input("Enter path to find the Decrypted files:- ")
                    decypt.get_files(directory)
                    r= decypt.start_decrypt()
                    if r=='SUCCESS':
                        print("\tDecrytpion Successful")
                    else:
                        print(r)
                elif(ch==3):
                    print("Note:-\n1.Do not change password if you already have encrypted file\nplease decrypt those files and change the password")
                    input("Press any key to continue.....")
                    count=3
                    while(count >0):

                        pscd=getpass.getpass("ENTER PASSWORD AGAIN...")
                        if(psw.load_password(pscd)==True):
                            change_pass()
                            break
                        else:

                            print("{} chance remines".format(count-1))
                            count=count-1      


                else:
                    print("INVALID ENTRY")


            else:
                print("PASSWORD DOES NOT MATCH")

        except Exception as e:
            print(e)

    else:
        change_pass()

    

if __name__=='__main__':
    main()



ENTER PASSWORD:- ········
***************************
WELCOME aniru
***************************
__________ MENU __________
1.Encrypt File
2.Decrypt File
3.Change Password
2
Enter path to find the Decrypted files:- G:\New folder
Files 1. :G:\New folder\DSC_0001.JPG.aes
Files 2. :G:\New folder\DSC_0002.JPG.aes
Files 3. :G:\New folder\DSC_0003.JPG.aes
Files 4. :G:\New folder\DSC_0004.JPG.aes
Files 5. :G:\New folder\DSC_0005.JPG.aes
Files 6. :G:\New folder\DSC_0006.JPG.aes
Files 7. :G:\New folder\DSC_0007.JPG.aes
Files 8. :G:\New folder\DSC_0008.JPG.aes
Files 9. :G:\New folder\DSC_0009.JPG.aes
want to decrypt all files..
1.YES
2.NO
1
wait...
G:\New folder\DSC_0001.JPG.aes
G:\New folder\DSC_0002.JPG.aes
G:\New folder\DSC_0003.JPG.aes
G:\New folder\DSC_0004.JPG.aes
G:\New folder\DSC_0005.JPG.aes
G:\New folder\DSC_0006.JPG.aes
G:\New folder\DSC_0007.JPG.aes
G:\New folder\DSC_0008.JPG.aes
G:\New folder\DSC_0009.JPG.aes
	Decrytpion Successful
